In [2]:
pip install pytube

  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
pip install youtube-transcript-api -q

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
from pytube import YouTube
from langchain_community.document_loaders import YoutubeLoader
link = 'https://www.youtube.com/watch?v=bSDprg24pEA'

def yt_title(link):
    try:
        print("Fetching title from:", link)
        loader = YoutubeLoader.from_youtube_url(link, add_video_info=True)
        docs = loader.load()
        if docs and 'title' in docs[0].metadata:
            return docs[0].metadata['title']
        return 'YouTube Video'
    except Exception as e:
        print("Failed to get title:", e)
        return 'YouTube Video'
    
print(yt_title(link))

Fetching title from: https://www.youtube.com/watch?v=bSDprg24pEA
Failed to get title: HTTP Error 400: Bad Request
YouTube Video


In [ ]:
import validators, streamlit as st
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import YoutubeLoader, UnstructuredURLLoader

st.set_page_config(page_title="LangChain: Summarize Text from YT videos or any Website", page_icon="hahaha")
st.title("LangChain: Summarize Text from YT videos or any Website")
st.subheader('Summarize URL')

with st.sidebar:
    groq_api_key = st.text_input("Groq API key", value="", type="password")

url = st.text_input("URL", label_visibility="collapsed")

# 🔽 Only run this after the button is clicked and inputs are valid
if st.button("Summarize the content fromYT or Website"):
    if not groq_api_key.strip() or not url.strip():
        st.error("Please provide the information")

    elif not validators.url(url):
        st.error("Please enter a valid URL")

    else:
        try:
            with st.spinner("Waiting..."):
                # ✅ Move inside valid logic
                llm = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

                if "youtube.com" in url:
                    loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
                else:
                    loader = UnstructuredURLLoader(
                        urls=[url],
                        ssl_verify=False,
                        headers={"User-Agent": "Mozilla/5.0"}
                    )

                docs = loader.load()

                # You missed defining this earlier
                prompt = PromptTemplate.from_template("Write a concise summary:\n{text}")

                chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
                output_summary = chain.run(docs)

                st.success(output_summary)
        except Exception as e:
            st.exception(e)

### 50 Correct Sentences

In [2]:
correct = '''
She walks to school every day.
I have finished my homework.
They were watching a movie last night.
He doesn't like spicy food.
The children are playing in the garden.
My brother has a black dog.
This book is very interesting.
We will go to the beach tomorrow.
I enjoy listening to music.
The cake was delicious.
She speaks English fluently.
I met him at the library yesterday.
Could you help me with this?
There is a cat under the table.
We have lived here for ten years.
It is raining outside.
He studies hard for his exams.
The teacher gave us homework.
I love watching the sunset.
This chair is made of wood.
She is wearing a red dress.
They arrived at the airport on time.
My father is reading the newspaper.
The sun rises in the east.
I would like a cup of coffee.
The train leaves at 6 p.m.
She was born in 1990.
He can swim very well.
We went hiking last weekend.
Please close the door.
I'm learning how to play the guitar.
The baby is sleeping in the crib.
He painted the house by himself.
I don't understand the question.
The flowers smell lovely.
I always take a walk after dinner.
She cooked dinner for the whole family.
They are watching a documentary.
The car is parked outside.
I need some help with my project.
She enjoys reading historical fiction.
He usually takes the bus to work.
The apples are in the basket.
I saw a shooting star last night.
My phone battery is low.
He is taller than his brother.
Let's meet at the coffee shop.
The birds are singing in the trees.
We cleaned the house yesterday.
I forgot my umbrella at home.
'''

### 50 Incorrect Sentences

In [3]:
incorrect = '''
She walk to school every day.
I has finished my homework.
They was watching a movie last night.
He don't like spicy food.
The childrens are playing in the garden.
My brother have a black dog.
This book very interesting.
We will goes to the beach tomorrow.
I enjoys listening to music.
The cake were delicious.
She speak English fluently.
I meet him at the library yesterday.
Could you helping me with this?
There are a cat under the table.
We have live here for ten years.
It raining outside.
He study hard for his exams.
The teacher give us homework.
I love watch the sunset.
This chair made of wood.
She wearing a red dress.
They arrived to the airport on time.
My father reading the newspaper.
The sun rise in the east.
I would likes a cup of coffee.
The train leave at 6 p.m.
She was born on 1990.
He can swims very well.
We gone hiking last weekend.
Please closes the door.
I'm learn how to play the guitar.
The baby sleep in the crib.
He paint the house by himself.
I doesn't understand the question.
The flowers smells lovely.
I always takes a walk after dinner.
She cooking dinner for the whole family.
They watching a documentary.
The car parked outside.
I needing some help with my project.
She enjoy reading historical fiction.
He usually take the bus to work.
The apples is in the basket.
I seed a shooting star last night.
My phone battery are low.
He is more taller than his brother.
Let's met at the coffee shop.
The birds singing in the trees.
We clean the house yesterday.
I forgetted my umbrella at home.
'''

In [4]:
import spacy
import pandas as pd

# 1. Loading the language library
nlp = spacy.load('en_core_web_sm')

### Each Token, POS, Dependency, Lemma

In [ ]:
# 2. Building a Pipline Object
doc = nlp(correct)

# 3. Using Tokens
for token in doc:
    print(f"{token.text:{12}}{token.pos_:{12}}{token.dep_:{12}}{token.lemma_}")


           SPACE       dep         

She         PRON        nsubj       she
walks       VERB        ROOT        walk
to          ADP         prep        to
school      NOUN        pobj        school
every       DET         det         every
day         NOUN        npadvmod    day
.           PUNCT       punct       .

           SPACE       dep         

I           PRON        nsubj       I
have        AUX         aux         have
finished    VERB        ROOT        finish
my          PRON        poss        my
homework    NOUN        dobj        homework
.           PUNCT       punct       .

           SPACE       dep         

They        PRON        nsubj       they
were        AUX         aux         be
watching    VERB        ROOT        watch
a           DET         det         a
movie       NOUN        dobj        movie
last        ADJ         amod        last
night       NOUN        npadvmod    night
.           PUNCT       punct       .

           SPACE       dep         

### The above sentences have very less number of named entities

In [8]:
for entity in doc.ents:
    print(f"{entity.text:-<{20}}{entity.label_:-<{20}}{str(spacy.explain(entity.label_))}")

last night----------TIME----------------Times smaller than a day
tomorrow------------DATE----------------Absolute or relative dates or periods
English-------------LANGUAGE------------Any named language
yesterday-----------DATE----------------Absolute or relative dates or periods
ten years-----------DATE----------------Absolute or relative dates or periods
6 p.m.--------------TIME----------------Times smaller than a day
1990----------------DATE----------------Absolute or relative dates or periods
last weekend--------DATE----------------Absolute or relative dates or periods
yesterday-----------DATE----------------Absolute or relative dates or periods


### Named Entity Recognition with new Data

In [2]:
ne ='''
1. Barack Obama was born in Hawaii in 1961.
2. Apple Inc. announced the iPhone 15 in September 2023.
3. I visited the Eiffel Tower in Paris last summer.
4. Microsoft released Windows 11 in October 2021.
5. Serena Williams won her last Grand Slam at the US Open.
6. Amazon’s headquarters is located in Seattle.
7. Elon Musk founded SpaceX in 2002.
8. The United Nations was established in 1945.
9. Lionel Messi plays for Inter Miami in the United States.
10. The meeting is scheduled for 3 p.m. on Tuesday.
11. Harvard University is one of the oldest institutions in the USA.
12. We stayed at the Marriott Hotel in Dubai.
13. Facebook changed its name to Meta in 2021.
14. They launched the new Tesla Model S in California.
15. The Taj Mahal is located in Agra, India.
16. NASA plans to send astronauts to Mars by 2030.
17. I booked my flight on Emirates Airlines to London.
18. Google Translate supports over 100 languages including French and Chinese.
19. She interned at Goldman Sachs last summer.
20. The New York Times published the article on Sunday.
21. Cristiano Ronaldo signed a contract with Al-Nassr FC.
22. We attended a conference in Berlin last week.
23. Shakespeare was born in Stratford-upon-Avon.
24. Toyota is the largest car manufacturer in Japan.
25. Jeff Bezos stepped down as CEO of Amazon in July 2021.
26. The Louvre Museum houses the Mona Lisa in Paris.
27. I watched the sunrise at Mount Fuji in Japan.
28. The Grand Canyon is located in Arizona, USA.
29. My appointment with Dr. Smith is at 9:00 a.m.
30. Uber operates in more than 60 countries worldwide.
31. IBM collaborated with MIT for quantum computing research.
32. Leonardo da Vinci painted The Last Supper in the 15th century.
33. My cousin studies at the University of Cambridge.
34. We traveled from Rome to Venice by train.
35. The Olympics will be held in Los Angeles in 2028.
36. Taylor Swift performed in Singapore last month.
37. The Bank of England raised interest rates in March.
38. Samsung launched the Galaxy S23 Ultra this year.
39. Adobe Photoshop is widely used by graphic designers.
40. I read The Great Gatsby by F. Scott Fitzgerald.
41. Intel unveiled its new AI chip in Las Vegas.
42. The President of France addressed the nation yesterday.
43. The river Nile flows through Egypt and Sudan.
44. My friend lives on Fifth Avenue in New York City.
45. We watched "Oppenheimer" at PVR Cinemas last Friday.
46. He started his job at Deloitte in August.
47. The Sydney Opera House is a world-famous landmark.
48. BMW is manufacturing electric cars in Germany.
49. We visited Niagara Falls on our honeymoon in June.
50. She donated $5,000 to the Red Cross after the earthquake.
'''

In [5]:
doc = nlp(ne)

In [ ]:
for entity in doc.ents:
    print(f"{entity.text:-<{20}}{entity.label_:-<{20}}{str(spacy.explain(entity.label_))}")

1-------------------CARDINAL------------Numerals that do not fall under another type
Barack Obama--------PERSON--------------People, including fictional
Hawaii--------------GPE-----------------Countries, cities, states
1961----------------DATE----------------Absolute or relative dates or periods
2-------------------CARDINAL------------Numerals that do not fall under another type
Apple Inc.----------ORG-----------------Companies, agencies, institutions, etc.
September 2023------DATE----------------Absolute or relative dates or periods
3-------------------CARDINAL------------Numerals that do not fall under another type
the Eiffel Tower----LOC-----------------Non-GPE locations, mountain ranges, bodies of water
Paris---------------GPE-----------------Countries, cities, states
last summer---------DATE----------------Absolute or relative dates or periods
4-------------------CARDINAL------------Numerals that do not fall under another type
Microsoft-----------ORG-----------------Companies, age

### NER with incorrect sentences

In [ ]:
ine = '''
1. Barack Obama borned was Hawaii in 1961.
2. Apple Inc. announce the iPhone 15 at September 2023.
3. I goes Eiffel Tower visit in Paris last summering.
4. Microsoft releasing Windows 11 on October 2021.
5. Serena Williams win she Grand Slam last at US Open.
6. Amazon headquarter are in Seattle is.
7. Elon Musk was founded SpaceX 2002.
8. United Nations be establish in year 1945.
9. Lionel Messi play with Inter Miami on United State.
10. The meeting scheduled be 3 p.m. on at Tuesday.
11. Harvard University oldest one institutions of USA.
12. We was stay Marriott Hotel to Dubai.
13. Facebook has change its named to Meta 2021 in.
14. They launch a new Tesla Model S California.
15. Taj Mahal in Agra located is India.
16. NASA plan send to astronauts Mars by 2030.
17. I book Emirates flight Airlines to for London.
18. Google Translate supports 100 over language like French with Chinese.
19. She interning Goldman Sachs summer last at.
20. New York Times article the on Sunday published.
21. Cristiano Ronaldo sign contract Al-Nassr FC with.
22. We conference in Berlin attending last week.
23. Shakespeare borned in Stratford-upon-Avon be.
24. Toyota largest car manufacturer in Japan is.
25. Jeff Bezos was step down Amazon CEO July 2021 in.
26. Louvre Museum Mona Lisa house Paris in.
27. I was sunrise watch Mount Fuji Japan at.
28. Grand Canyon be Arizona USA located.
29. My appointment Dr. Smith 9:00 a.m. is on.
30. Uber operate than 60 more countries over.
31. IBM MIT with collaborating quantum compute.
32. Leonardo da Vinci the Last Supper painting century 15.
33. My cousin do study University Cambridge at.
34. We was traveling train with Rome to Venice.
35. Olympics holding will Los Angeles at 2028.
36. Taylor Swift last Singapore perform in month.
37. Bank England of rise rates March interest.
38. Samsung Galaxy S23 Ultra launched was this year in.
39. Adobe Photoshop use many for designer graphic.
40. I Gatsby Great read by F. Scott Fitzgerald.
41. Intel unveiled Las Vegas AI chip in new its.
42. France President yesterday nation the address.
43. River Nile go through Egypt and Sudan flow.
44. My friend Fifth Avenue on live in New York.
45. We “Oppenheimer” seen PVR Cinemas Friday last.
46. He job started Deloitte his in August at.
47. Sydney Opera House landmark world-famous is.
48. BMW electric cars make Germany in are.
49. We Niagara Falls June honeymoon on visited.
50. She was donate $5,000 for Red Cross the earthquake after.
'''

In [13]:
doc = nlp(ine)
for entity in doc.ents:
    print(f"{entity.text:-<{20}}{entity.label_:-<{20}}{str(spacy.explain(entity.label_))}")

1-------------------CARDINAL------------Numerals that do not fall under another type
Barack Obama--------PERSON--------------People, including fictional
Hawaii--------------GPE-----------------Countries, cities, states
1961----------------DATE----------------Absolute or relative dates or periods
2-------------------CARDINAL------------Numerals that do not fall under another type
Apple Inc.----------ORG-----------------Companies, agencies, institutions, etc.
September 2023------DATE----------------Absolute or relative dates or periods
3-------------------CARDINAL------------Numerals that do not fall under another type
Eiffel Tower--------FAC-----------------Buildings, airports, highways, bridges, etc.
Paris---------------GPE-----------------Countries, cities, states
4-------------------CARDINAL------------Numerals that do not fall under another type
Microsoft-----------ORG-----------------Companies, agencies, institutions, etc.
Windows 11----------PRODUCT-------------Objects, vehicles, 

### Noun Chunking

A noun chunk is a base noun phrase — that means it's:

A noun and all its modifiers,

But not including other noun phrases or clauses.

In [14]:
for chunk in doc.noun_chunks:
    print(chunk.text)

Barack Obama
Hawaii
Apple Inc.
the iPhone
September
I
Eiffel Tower visit
Paris
Microsoft
Windows
October
Serena Williams
she
Grand Slam
US Open
Seattle
Elon Musk
SpaceX
United Nations
year
Lionel Messi
Inter Miami
United State
The meeting
3 p.m.
Tuesday
Harvard University
oldest one institutions
USA
We
Marriott Hotel
Dubai
Facebook
Meta
They
a new Tesla Model S California
Taj Mahal
Agra
India
astronauts
Mars
I
Emirates flight Airlines
London
Google Translate
language
French
Chinese
She
Goldman Sachs summer
New York Times article
the
Sunday
Cristiano Ronaldo sign contract
Al-Nassr FC
We
Berlin
Shakespeare
Avon
Toyota largest car manufacturer
Japan
Jeff Bezos
step
Amazon CEO
Louvre Museum Mona Lisa house
Paris
I
Mount Fuji Japan
Grand Canyon
Arizona USA
My appointment
Dr. Smith
than 60 more countries
IBM MIT
quantum compute
Leonardo da Vinci
century
My cousin
University Cambridge
We
train
Rome
Venice
Olympics
Los Angeles
Taylor Swift
last Singapore
month
Bank England
rise rates
Samsung G

In [ ]:
D:\CODING\AAA_Violence Detection\data\4A-Dataset-for-Automatic-Violence-Detection-in-Videos\A-Dataset-for-Automatic-Violence-Detection-in-Videos-master\violence-detection-dataset\non-violent\cam2

In [3]:
import os

# Path to your cam2 folder
folder_path = r'D:\CODING\AAA_Violence Detection\data\4A-Dataset-for-Automatic-Violence-Detection-in-Videos\A-Dataset-for-Automatic-Violence-Detection-in-Videos-master\violence-detection-dataset\violent\cam2'

# Get all files in the folder and sort them
files = sorted(os.listdir(folder_path))

# Start renaming from 61
start_number = 236

for filename in files:
    file_path = os.path.join(folder_path, filename)
    
    if os.path.isfile(file_path):
        # Get file extension
        _, ext = os.path.splitext(filename)
        # Generate new name
        new_name = f"{start_number}{ext}"
        new_path = os.path.join(folder_path, new_name)
        
        # Rename the file
        os.rename(file_path, new_path)
        print(f"Renamed: {filename} -> {new_name}")
        start_number += 1


Renamed: 1.mp4 -> 236.mp4
Renamed: 10.mp4 -> 237.mp4
Renamed: 100.mp4 -> 238.mp4
Renamed: 101.mp4 -> 239.mp4
Renamed: 102.mp4 -> 240.mp4
Renamed: 103.mp4 -> 241.mp4
Renamed: 104.mp4 -> 242.mp4
Renamed: 105.mp4 -> 243.mp4
Renamed: 106.mp4 -> 244.mp4
Renamed: 107.mp4 -> 245.mp4
Renamed: 108.mp4 -> 246.mp4
Renamed: 109.mp4 -> 247.mp4
Renamed: 11.mp4 -> 248.mp4
Renamed: 110.mp4 -> 249.mp4
Renamed: 111.mp4 -> 250.mp4
Renamed: 112.mp4 -> 251.mp4
Renamed: 113.mp4 -> 252.mp4
Renamed: 114.mp4 -> 253.mp4
Renamed: 115.mp4 -> 254.mp4
Renamed: 12.mp4 -> 255.mp4
Renamed: 13.mp4 -> 256.mp4
Renamed: 14.mp4 -> 257.mp4
Renamed: 15.mp4 -> 258.mp4
Renamed: 16.mp4 -> 259.mp4
Renamed: 17.mp4 -> 260.mp4
Renamed: 18.mp4 -> 261.mp4
Renamed: 19.mp4 -> 262.mp4
Renamed: 2.mp4 -> 263.mp4
Renamed: 20.mp4 -> 264.mp4
Renamed: 21.mp4 -> 265.mp4
Renamed: 22.mp4 -> 266.mp4
Renamed: 23.mp4 -> 267.mp4
Renamed: 24.mp4 -> 268.mp4
Renamed: 25.mp4 -> 269.mp4
Renamed: 26.mp4 -> 270.mp4
Renamed: 27.mp4 -> 271.mp4
Renamed: 28.mp